# Phase 6B: Fully Integrated Quantum-Enhanced World Model

**Quantum-Enhanced Simulation Learning for Reinforcement Learning**

Author: Saurabh Jalendra  
Institution: BITS Pilani (WILP Division)  
Date: December 2025

---

## Overview

This notebook implements the **Fully Integrated** approach that combines ALL quantum-inspired components:

| Component | Source | Target Bottleneck |
|-----------|--------|-------------------|
| QAOA Optimizer | Notebook 03 | Local Minima |
| Superposition Replay | Notebook 04 | Sample Inefficiency |
| Gate-Enhanced Layers | Notebook 05 | Slow Convergence |
| Error Correction Ensemble | Notebook 06 | Compounding Errors |

**Research Question:** Does combining all quantum-inspired methods provide better results than any single method alone?

## 6B.1 Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gymnasium as gym
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass
from collections import defaultdict
import matplotlib.pyplot as plt
from pathlib import Path
import json
import time
from scipy import stats

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')

# Standard configuration (consistent with all notebooks)
CONFIG = {
    'obs_dim': 4,
    'action_dim': 2,
    'stoch_dim': 64,
    'deter_dim': 512,
    'hidden_dim': 512,
    'batch_size': 32,
    'seq_len': 20,
    'num_epochs': 50,
    'learning_rate': 3e-4,
    'num_episodes': 100,
    'num_ensemble': 5,
    'seeds': [42, 123, 456, 789, 1024],
}

print(f'Configuration: {CONFIG}')

## 6B.2 Component 1: Gate-Enhanced Layers

Quantum gate-inspired neural network layers for richer representations.

In [ ]:
class HadamardLayer(nn.Module):
    """Hadamard-inspired feature mixing layer."""
    
    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim
        # Learnable Hadamard-like transformation
        self.weight = nn.Parameter(torch.randn(dim, dim) / np.sqrt(dim))
        self.bias = nn.Parameter(torch.zeros(dim))
        # Make it close to orthogonal
        nn.init.orthogonal_(self.weight)
    
    def forward(self, x):
        # Apply Hadamard-like mixing
        return F.linear(x, self.weight, self.bias)


class PhaseLayer(nn.Module):
    """Phase gate-inspired modulation layer."""
    
    def __init__(self, dim: int):
        super().__init__()
        self.phase = nn.Parameter(torch.zeros(dim))
    
    def forward(self, x):
        # Apply learnable phase modulation
        return x * torch.cos(self.phase) + torch.roll(x, 1, dims=-1) * torch.sin(self.phase)


class EntanglementLayer(nn.Module):
    """CNOT-inspired entanglement layer."""
    
    def __init__(self, dim: int, num_pairs: int = 4):
        super().__init__()
        self.dim = dim
        self.num_pairs = min(num_pairs, dim // 2)
        self.control_weights = nn.Parameter(torch.randn(self.num_pairs))
    
    def forward(self, x):
        output = x.clone()
        for i in range(self.num_pairs):
            control_idx = i * 2
            target_idx = i * 2 + 1
            if target_idx < self.dim:
                # CNOT-like: target XOR control (soft version)
                gate = torch.sigmoid(self.control_weights[i])
                output[..., target_idx] = output[..., target_idx] * (1 - gate) + \
                                          (output[..., target_idx] * output[..., control_idx]) * gate
        return output


class QuantumGateBlock(nn.Module):
    """Combined quantum gate block with all gate types."""
    
    def __init__(self, dim: int, use_hadamard: bool = True, use_phase: bool = True, use_entangle: bool = True):
        super().__init__()
        layers = []
        if use_hadamard:
            layers.append(HadamardLayer(dim))
            layers.append(nn.ELU())
        if use_phase:
            layers.append(PhaseLayer(dim))
        if use_entangle:
            layers.append(EntanglementLayer(dim))
        layers.append(nn.LayerNorm(dim))
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x) + x  # Residual connection

print('Gate-Enhanced Layers defined.')

## 6B.3 Component 2: Gate-Enhanced World Model

World model with quantum gate layers integrated into encoder, decoder, and RSSM.

In [ ]:
class GateEnhancedWorldModel(nn.Module):
    """World model with quantum gate-enhanced layers."""
    
    def __init__(self, obs_dim: int, action_dim: int, config: Dict):
        super().__init__()
        self.obs_dim = obs_dim
        self.action_dim = action_dim
        self.stoch_dim = config.get('stoch_dim', 64)
        self.deter_dim = config.get('deter_dim', 512)
        self.hidden_dim = config.get('hidden_dim', 512)
        self.state_dim = self.stoch_dim + self.deter_dim
        
        # Gate-enhanced encoder
        self.encoder = nn.Sequential(
            nn.Linear(obs_dim, self.hidden_dim),
            nn.ELU(),
            QuantumGateBlock(self.hidden_dim),
            nn.Linear(self.hidden_dim, self.hidden_dim // 2),
            nn.ELU(),
        )
        
        # RSSM components with gate enhancement
        self.gru = nn.GRUCell(self.stoch_dim + action_dim, self.deter_dim)
        
        self.prior = nn.Sequential(
            nn.Linear(self.deter_dim, self.hidden_dim // 2),
            nn.ELU(),
            QuantumGateBlock(self.hidden_dim // 2),
            nn.Linear(self.hidden_dim // 2, self.stoch_dim * 2)
        )
        
        self.posterior = nn.Sequential(
            nn.Linear(self.deter_dim + self.hidden_dim // 2, self.hidden_dim // 2),
            nn.ELU(),
            QuantumGateBlock(self.hidden_dim // 2),
            nn.Linear(self.hidden_dim // 2, self.stoch_dim * 2)
        )
        
        # Gate-enhanced decoder
        self.decoder = nn.Sequential(
            nn.Linear(self.state_dim, self.hidden_dim),
            nn.ELU(),
            QuantumGateBlock(self.hidden_dim),
            nn.Linear(self.hidden_dim, obs_dim * 2)
        )
        
        # Reward predictor
        self.reward_pred = nn.Sequential(
            nn.Linear(self.state_dim, self.hidden_dim // 2),
            nn.ELU(),
            nn.Linear(self.hidden_dim // 2, 1)
        )
    
    def initial_state(self, batch_size: int):
        return {
            'deter': torch.zeros(batch_size, self.deter_dim, device=DEVICE),
            'stoch': torch.zeros(batch_size, self.stoch_dim, device=DEVICE)
        }
    
    def get_dist(self, stats):
        mean, log_std = stats.chunk(2, dim=-1)
        std = F.softplus(log_std) + 0.1
        return torch.distributions.Normal(mean, std)
    
    def forward(self, obs_seq, action_seq):
        batch_size, seq_len = obs_seq.shape[:2]
        
        # Initialize
        state = self.initial_state(batch_size)
        
        # Collect outputs
        prior_means, prior_stds = [], []
        post_means, post_stds = [], []
        obs_means, obs_stds = [], []
        rewards = []
        states = []
        
        for t in range(seq_len):
            # Encode observation
            embed = self.encoder(obs_seq[:, t])
            
            # Prior (before seeing observation)
            prior_stats = self.prior(state['deter'])
            prior_dist = self.get_dist(prior_stats)
            
            # Posterior (after seeing observation)
            post_input = torch.cat([state['deter'], embed], dim=-1)
            post_stats = self.posterior(post_input)
            post_dist = self.get_dist(post_stats)
            
            # Sample stochastic state
            stoch = post_dist.rsample()
            
            # Update deterministic state
            gru_input = torch.cat([stoch, action_seq[:, t]], dim=-1)
            deter = self.gru(gru_input, state['deter'])
            
            # Full state
            full_state = torch.cat([deter, stoch], dim=-1)
            
            # Decode
            obs_stats = self.decoder(full_state)
            obs_dist = self.get_dist(obs_stats)
            
            # Reward
            reward = self.reward_pred(full_state)
            
            # Store
            prior_means.append(prior_dist.mean)
            prior_stds.append(prior_dist.stddev)
            post_means.append(post_dist.mean)
            post_stds.append(post_dist.stddev)
            obs_means.append(obs_dist.mean)
            obs_stds.append(obs_dist.stddev)
            rewards.append(reward)
            states.append(full_state)
            
            # Update state
            state = {'deter': deter, 'stoch': stoch}
        
        return {
            'prior_mean': torch.stack(prior_means, dim=1),
            'prior_std': torch.stack(prior_stds, dim=1),
            'post_mean': torch.stack(post_means, dim=1),
            'post_std': torch.stack(post_stds, dim=1),
            'obs_mean': torch.stack(obs_means, dim=1),
            'obs_std': torch.stack(obs_stds, dim=1),
            'reward': torch.stack(rewards, dim=1).squeeze(-1),
            'states': torch.stack(states, dim=1),
        }

# Test
model = GateEnhancedWorldModel(CONFIG['obs_dim'], CONFIG['action_dim'], CONFIG).to(DEVICE)
print(f'GateEnhancedWorldModel parameters: {sum(p.numel() for p in model.parameters()):,}')

## 6B.4 Component 3: Error Correction Ensemble

Ensemble of gate-enhanced models with majority voting for robust predictions.

In [ ]:
class ErrorCorrectionEnsemble(nn.Module):
    """Ensemble of gate-enhanced world models with error correction."""
    
    def __init__(self, obs_dim: int, action_dim: int, config: Dict, num_models: int = 5):
        super().__init__()
        self.num_models = num_models
        self.models = nn.ModuleList([
            GateEnhancedWorldModel(obs_dim, action_dim, config)
            for _ in range(num_models)
        ])
        self.correction_method = config.get('correction_method', 'weighted')
    
    def forward(self, obs_seq, action_seq, return_all: bool = False):
        # Get predictions from all models
        all_outputs = [model(obs_seq, action_seq) for model in self.models]
        
        # Stack predictions
        obs_preds = torch.stack([out['obs_mean'] for out in all_outputs], dim=0)
        reward_preds = torch.stack([out['reward'] for out in all_outputs], dim=0)
        
        # Apply error correction
        if self.correction_method == 'majority':
            corrected_obs = obs_preds.median(dim=0).values
            corrected_reward = reward_preds.median(dim=0).values
        elif self.correction_method == 'weighted':
            # Weight by inverse disagreement
            disagreement = obs_preds.var(dim=0, keepdim=True)
            weights = 1.0 / (disagreement.mean(dim=-1, keepdim=True) + 1e-6)
            weights = weights / weights.sum(dim=0, keepdim=True)
            corrected_obs = (obs_preds * weights).sum(dim=0)
            corrected_reward = (reward_preds * weights.squeeze(-1)).sum(dim=0)
        else:  # simple average
            corrected_obs = obs_preds.mean(dim=0)
            corrected_reward = reward_preds.mean(dim=0)
        
        # Calculate uncertainty
        uncertainty = obs_preds.std(dim=0).mean(dim=-1)
        
        result = {
            'obs_mean': corrected_obs,
            'reward': corrected_reward,
            'uncertainty': uncertainty,
            'prior_mean': all_outputs[0]['prior_mean'],
            'prior_std': all_outputs[0]['prior_std'],
            'post_mean': all_outputs[0]['post_mean'],
            'post_std': all_outputs[0]['post_std'],
            'obs_std': all_outputs[0]['obs_std'],
        }
        
        if return_all:
            result['all_outputs'] = all_outputs
        
        return result

# Test
ensemble = ErrorCorrectionEnsemble(CONFIG['obs_dim'], CONFIG['action_dim'], CONFIG, num_models=5).to(DEVICE)
print(f'ErrorCorrectionEnsemble total parameters: {sum(p.numel() for p in ensemble.parameters()):,}')

## 6B.5 Component 4: Superposition Replay Buffer

Experience replay with quantum superposition-inspired sampling.

In [ ]:
@dataclass
class Episode:
    observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    total_reward: float


class SuperpositionReplayBuffer:
    """Replay buffer with quantum superposition-inspired sampling."""
    
    def __init__(self, capacity: int = 1000, parallel_samples: int = 4):
        self.capacity = capacity
        self.parallel_samples = parallel_samples
        self.episodes: List[Episode] = []
        self.amplitudes: np.ndarray = np.array([])
    
    def add_episode(self, episode: Episode):
        if len(self.episodes) >= self.capacity:
            self.episodes.pop(0)
        self.episodes.append(episode)
        self._update_amplitudes()
    
    def _update_amplitudes(self):
        """Update quantum-like amplitudes based on episode quality."""
        if not self.episodes:
            self.amplitudes = np.array([])
            return
        
        rewards = np.array([ep.total_reward for ep in self.episodes])
        # Normalize to create amplitude distribution
        if rewards.std() > 0:
            normalized = (rewards - rewards.mean()) / rewards.std()
        else:
            normalized = np.zeros_like(rewards)
        
        # Convert to probabilities (amplitude squared)
        amplitudes = np.exp(normalized)
        self.amplitudes = amplitudes / amplitudes.sum()
    
    def sample_superposition(self, batch_size: int, seq_len: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Sample using superposition-inspired parallel sampling."""
        if len(self.episodes) < self.parallel_samples:
            return self.sample_standard(batch_size, seq_len)
        
        # Sample multiple episodes in parallel (superposition)
        obs_batch, act_batch, rew_batch = [], [], []
        
        for _ in range(batch_size):
            # Sample parallel_samples episodes based on amplitudes
            indices = np.random.choice(
                len(self.episodes), 
                size=self.parallel_samples,
                p=self.amplitudes,
                replace=True
            )
            
            # "Collapse" to one by weighted combination
            selected_amplitudes = self.amplitudes[indices]
            collapse_probs = selected_amplitudes / selected_amplitudes.sum()
            chosen_idx = np.random.choice(indices, p=collapse_probs)
            
            episode = self.episodes[chosen_idx]
            
            # Random start point
            max_start = max(0, len(episode.observations) - seq_len)
            start = np.random.randint(0, max_start + 1) if max_start > 0 else 0
            end = min(start + seq_len, len(episode.observations))
            
            obs = episode.observations[start:end]
            act = episode.actions[start:end]
            rew = episode.rewards[start:end]
            
            # Pad if necessary
            if len(obs) < seq_len:
                pad_len = seq_len - len(obs)
                obs = np.pad(obs, ((0, pad_len), (0, 0)), mode='edge')
                act = np.pad(act, ((0, pad_len), (0, 0)), mode='edge')
                rew = np.pad(rew, (0, pad_len), mode='edge')
            
            obs_batch.append(obs)
            act_batch.append(act)
            rew_batch.append(rew)
        
        return (
            torch.FloatTensor(np.array(obs_batch)).to(DEVICE),
            torch.FloatTensor(np.array(act_batch)).to(DEVICE),
            torch.FloatTensor(np.array(rew_batch)).to(DEVICE)
        )
    
    def sample_standard(self, batch_size: int, seq_len: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Standard uniform sampling."""
        obs_batch, act_batch, rew_batch = [], [], []
        
        for _ in range(batch_size):
            episode = np.random.choice(self.episodes)
            max_start = max(0, len(episode.observations) - seq_len)
            start = np.random.randint(0, max_start + 1) if max_start > 0 else 0
            end = min(start + seq_len, len(episode.observations))
            
            obs = episode.observations[start:end]
            act = episode.actions[start:end]
            rew = episode.rewards[start:end]
            
            if len(obs) < seq_len:
                pad_len = seq_len - len(obs)
                obs = np.pad(obs, ((0, pad_len), (0, 0)), mode='edge')
                act = np.pad(act, ((0, pad_len), (0, 0)), mode='edge')
                rew = np.pad(rew, (0, pad_len), mode='edge')
            
            obs_batch.append(obs)
            act_batch.append(act)
            rew_batch.append(rew)
        
        return (
            torch.FloatTensor(np.array(obs_batch)).to(DEVICE),
            torch.FloatTensor(np.array(act_batch)).to(DEVICE),
            torch.FloatTensor(np.array(rew_batch)).to(DEVICE)
        )
    
    def __len__(self):
        return len(self.episodes)

print('SuperpositionReplayBuffer defined.')

## 6B.6 Component 5: QAOA-Inspired Optimizer

Optimizer that alternates between cost and mixing operators to escape local minima.

In [ ]:
class QAOAOptimizer:
    """QAOA-inspired optimizer with alternating cost/mixing phases."""
    
    def __init__(self, params, lr: float = 3e-4, p_layers: int = 4,
                 gamma_init: float = 0.1, beta_init: float = 0.001):
        self.params = list(params)
        self.lr = lr
        self.p_layers = p_layers
        
        # QAOA parameters
        self.gammas = [gamma_init] * p_layers
        self.betas = [beta_init] * p_layers
        
        # Base optimizer
        self.base_optimizer = torch.optim.Adam(self.params, lr=lr)
        
        # State tracking
        self.step_count = 0
        self.current_layer = 0
        self.phase = 'cost'  # 'cost' or 'mixing'
    
    def zero_grad(self):
        self.base_optimizer.zero_grad()
    
    def step(self, loss: torch.Tensor = None):
        """Perform QAOA-inspired optimization step."""
        self.step_count += 1
        
        # Determine current phase
        layer_idx = self.current_layer % self.p_layers
        
        if self.phase == 'cost':
            # Cost phase: standard gradient descent with gamma scaling
            gamma = self.gammas[layer_idx]
            for param in self.params:
                if param.grad is not None:
                    param.grad.data.mul_(gamma)
            self.base_optimizer.step()
            self.phase = 'mixing'
        else:
            # Mixing phase: add exploration noise
            beta = self.betas[layer_idx]
            with torch.no_grad():
                for param in self.params:
                    if param.requires_grad:
                        noise = torch.randn_like(param) * beta * self.lr
                        param.add_(noise)
            self.phase = 'cost'
            self.current_layer += 1
    
    def state_dict(self):
        return {
            'base_optimizer': self.base_optimizer.state_dict(),
            'step_count': self.step_count,
            'gammas': self.gammas,
            'betas': self.betas,
        }
    
    def load_state_dict(self, state_dict):
        self.base_optimizer.load_state_dict(state_dict['base_optimizer'])
        self.step_count = state_dict['step_count']
        self.gammas = state_dict['gammas']
        self.betas = state_dict['betas']

print('QAOAOptimizer defined.')

## 6B.7 Fully Integrated Trainer

Training loop that combines all quantum-inspired components.

In [ ]:
class FullyIntegratedTrainer:
    """
    Fully integrated trainer combining:
    - QAOA optimizer (escapes local minima)
    - Superposition replay (sample efficiency)
    - Gate-enhanced layers (better representations)
    - Error correction ensemble (robust predictions)
    """
    
    def __init__(self, ensemble: ErrorCorrectionEnsemble, buffer: SuperpositionReplayBuffer,
                 config: Dict):
        self.ensemble = ensemble
        self.buffer = buffer
        self.config = config
        
        # Create QAOA optimizer for each model in ensemble
        self.optimizers = [
            QAOAOptimizer(
                model.parameters(),
                lr=config.get('learning_rate', 3e-4),
                p_layers=config.get('qaoa_layers', 4),
                gamma_init=config.get('gamma_init', 0.1),
                beta_init=config.get('beta_init', 0.001)
            )
            for model in ensemble.models
        ]
        
        self.training_history = defaultdict(list)
    
    def compute_loss(self, outputs: Dict, obs_seq: torch.Tensor, reward_seq: torch.Tensor) -> Tuple[torch.Tensor, Dict]:
        """Compute world model loss."""
        # Reconstruction loss
        recon_loss = F.mse_loss(outputs['obs_mean'], obs_seq)
        
        # KL divergence
        prior_dist = torch.distributions.Normal(outputs['prior_mean'], outputs['prior_std'])
        post_dist = torch.distributions.Normal(outputs['post_mean'], outputs['post_std'])
        kl_loss = torch.distributions.kl_divergence(post_dist, prior_dist).mean()
        kl_loss = torch.clamp(kl_loss, min=1.0)  # Free bits
        
        # Reward loss
        reward_loss = F.mse_loss(outputs['reward'], reward_seq)
        
        # Total loss
        total_loss = recon_loss + 0.1 * kl_loss + reward_loss
        
        metrics = {
            'recon_loss': recon_loss.item(),
            'kl_loss': kl_loss.item(),
            'reward_loss': reward_loss.item(),
            'total_loss': total_loss.item(),
        }
        
        return total_loss, metrics
    
    def train_step(self, use_superposition: bool = True) -> Dict:
        """Single training step with all quantum components."""
        self.ensemble.train()
        
        # Sample using superposition replay
        if use_superposition:
            obs_seq, action_seq, reward_seq = self.buffer.sample_superposition(
                self.config['batch_size'], self.config['seq_len']
            )
        else:
            obs_seq, action_seq, reward_seq = self.buffer.sample_standard(
                self.config['batch_size'], self.config['seq_len']
            )
        
        total_loss = 0.0
        all_metrics = defaultdict(list)
        
        # Train each model in ensemble with QAOA optimizer
        for model, optimizer in zip(self.ensemble.models, self.optimizers):
            optimizer.zero_grad()
            
            outputs = model(obs_seq, action_seq)
            loss, metrics = self.compute_loss(outputs, obs_seq, reward_seq)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 100.0)
            optimizer.step(loss)
            
            total_loss += loss.item()
            for k, v in metrics.items():
                all_metrics[k].append(v)
        
        # Average metrics
        avg_metrics = {k: np.mean(v) for k, v in all_metrics.items()}
        avg_metrics['ensemble_loss'] = total_loss / len(self.ensemble.models)
        
        return avg_metrics
    
    def train(self, num_epochs: int, use_superposition: bool = True) -> Dict:
        """Full training loop."""
        start_time = time.time()
        
        for epoch in range(num_epochs):
            epoch_metrics = defaultdict(list)
            
            # Multiple steps per epoch
            num_steps = max(1, len(self.buffer) // self.config['batch_size'])
            for _ in range(num_steps):
                metrics = self.train_step(use_superposition)
                for k, v in metrics.items():
                    epoch_metrics[k].append(v)
            
            # Average epoch metrics
            for k, v in epoch_metrics.items():
                self.training_history[k].append(np.mean(v))
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}: Loss = {self.training_history['total_loss'][-1]:.4f}")
        
        training_time = time.time() - start_time
        self.training_history['training_time'] = training_time
        
        return dict(self.training_history)
    
    def evaluate(self, test_obs: torch.Tensor, test_actions: torch.Tensor) -> Dict:
        """Evaluate the ensemble."""
        self.ensemble.eval()
        with torch.no_grad():
            outputs = self.ensemble(test_obs, test_actions, return_all=True)
            
            # Ensemble prediction error
            ensemble_mse = F.mse_loss(outputs['obs_mean'], test_obs).item()
            
            # Individual model errors
            individual_mses = []
            for out in outputs['all_outputs']:
                mse = F.mse_loss(out['obs_mean'], test_obs).item()
                individual_mses.append(mse)
            
            # Uncertainty
            mean_uncertainty = outputs['uncertainty'].mean().item()
        
        return {
            'ensemble_mse': ensemble_mse,
            'avg_individual_mse': np.mean(individual_mses),
            'best_individual_mse': min(individual_mses),
            'worst_individual_mse': max(individual_mses),
            'mean_uncertainty': mean_uncertainty,
        }

print('FullyIntegratedTrainer defined.')

## 6B.8 Long-Horizon Prediction Test

Test multi-step prediction accuracy (critical for world models used in planning).

In [ ]:
def evaluate_long_horizon(model, obs_seq: torch.Tensor, action_seq: torch.Tensor, 
                          horizons: List[int] = [1, 5, 10, 20]) -> Dict:
    """
    Evaluate prediction accuracy at different horizons.
    
    This is critical because world models are used for PLANNING,
    which requires accurate predictions many steps into the future.
    """
    model.eval()
    results = {}
    
    with torch.no_grad():
        # Get model predictions
        if hasattr(model, 'models'):  # Ensemble
            outputs = model(obs_seq, action_seq)
            pred_obs = outputs['obs_mean']
        else:  # Single model
            outputs = model(obs_seq, action_seq)
            pred_obs = outputs['obs_mean']
        
        # Calculate error at each horizon
        for h in horizons:
            if h <= obs_seq.shape[1]:
                # MSE at horizon h
                mse_at_h = F.mse_loss(pred_obs[:, h-1:h], obs_seq[:, h-1:h]).item()
                results[f'mse_horizon_{h}'] = mse_at_h
        
        # Average error over all horizons
        results['avg_horizon_mse'] = np.mean([v for k, v in results.items() if 'mse_horizon' in k])
        
        # Error growth rate (how fast does error compound?)
        if len(horizons) >= 2:
            first_h = horizons[0]
            last_h = min(horizons[-1], obs_seq.shape[1])
            if f'mse_horizon_{first_h}' in results and f'mse_horizon_{last_h}' in results:
                error_growth = results[f'mse_horizon_{last_h}'] / (results[f'mse_horizon_{first_h}'] + 1e-8)
                results['error_growth_rate'] = error_growth
    
    return results

print('Long-horizon evaluation function defined.')

## 6B.9 Data Collection

In [ ]:
def collect_episodes(env_name: str, num_episodes: int, seed: int = 42) -> List[Episode]:
    """Collect episodes from environment."""
    env = gym.make(env_name)
    episodes = []
    
    for ep in range(num_episodes):
        obs, _ = env.reset(seed=seed + ep)
        observations, actions, rewards = [obs], [], []
        
        done = False
        while not done:
            action = env.action_space.sample()
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # One-hot encode action for discrete spaces
            if isinstance(env.action_space, gym.spaces.Discrete):
                action_onehot = np.zeros(env.action_space.n)
                action_onehot[action] = 1
                actions.append(action_onehot)
            else:
                actions.append(action)
            
            rewards.append(reward)
            observations.append(next_obs)
        
        episodes.append(Episode(
            observations=np.array(observations[:-1]),
            actions=np.array(actions),
            rewards=np.array(rewards),
            total_reward=sum(rewards)
        ))
    
    env.close()
    avg_reward = np.mean([ep.total_reward for ep in episodes])
    avg_length = np.mean([len(ep.observations) for ep in episodes])
    print(f'Collected {num_episodes} episodes, avg reward: {avg_reward:.1f}, avg length: {avg_length:.1f}')
    return episodes

# Collect data
print('Collecting training data...')
episodes = collect_episodes('CartPole-v1', CONFIG['num_episodes'], seed=42)

## 6B.10 Classical Baseline for Comparison

In [ ]:
class ClassicalWorldModel(nn.Module):
    """Standard world model without quantum enhancements (for comparison)."""
    
    def __init__(self, obs_dim: int, action_dim: int, config: Dict):
        super().__init__()
        self.obs_dim = obs_dim
        self.action_dim = action_dim
        self.stoch_dim = config.get('stoch_dim', 64)
        self.deter_dim = config.get('deter_dim', 512)
        self.hidden_dim = config.get('hidden_dim', 512)
        self.state_dim = self.stoch_dim + self.deter_dim
        
        # Standard encoder (no gates)
        self.encoder = nn.Sequential(
            nn.Linear(obs_dim, self.hidden_dim),
            nn.ELU(),
            nn.Linear(self.hidden_dim, self.hidden_dim // 2),
            nn.ELU(),
        )
        
        # Standard RSSM
        self.gru = nn.GRUCell(self.stoch_dim + action_dim, self.deter_dim)
        
        self.prior = nn.Sequential(
            nn.Linear(self.deter_dim, self.hidden_dim // 2),
            nn.ELU(),
            nn.Linear(self.hidden_dim // 2, self.stoch_dim * 2)
        )
        
        self.posterior = nn.Sequential(
            nn.Linear(self.deter_dim + self.hidden_dim // 2, self.hidden_dim // 2),
            nn.ELU(),
            nn.Linear(self.hidden_dim // 2, self.stoch_dim * 2)
        )
        
        # Standard decoder
        self.decoder = nn.Sequential(
            nn.Linear(self.state_dim, self.hidden_dim),
            nn.ELU(),
            nn.Linear(self.hidden_dim, obs_dim * 2)
        )
        
        self.reward_pred = nn.Sequential(
            nn.Linear(self.state_dim, self.hidden_dim // 2),
            nn.ELU(),
            nn.Linear(self.hidden_dim // 2, 1)
        )
    
    def initial_state(self, batch_size: int):
        return {
            'deter': torch.zeros(batch_size, self.deter_dim, device=DEVICE),
            'stoch': torch.zeros(batch_size, self.stoch_dim, device=DEVICE)
        }
    
    def get_dist(self, stats):
        mean, log_std = stats.chunk(2, dim=-1)
        std = F.softplus(log_std) + 0.1
        return torch.distributions.Normal(mean, std)
    
    def forward(self, obs_seq, action_seq):
        batch_size, seq_len = obs_seq.shape[:2]
        state = self.initial_state(batch_size)
        
        prior_means, prior_stds = [], []
        post_means, post_stds = [], []
        obs_means, obs_stds = [], []
        rewards = []
        
        for t in range(seq_len):
            embed = self.encoder(obs_seq[:, t])
            
            prior_stats = self.prior(state['deter'])
            prior_dist = self.get_dist(prior_stats)
            
            post_input = torch.cat([state['deter'], embed], dim=-1)
            post_stats = self.posterior(post_input)
            post_dist = self.get_dist(post_stats)
            
            stoch = post_dist.rsample()
            gru_input = torch.cat([stoch, action_seq[:, t]], dim=-1)
            deter = self.gru(gru_input, state['deter'])
            full_state = torch.cat([deter, stoch], dim=-1)
            
            obs_stats = self.decoder(full_state)
            obs_dist = self.get_dist(obs_stats)
            reward = self.reward_pred(full_state)
            
            prior_means.append(prior_dist.mean)
            prior_stds.append(prior_dist.stddev)
            post_means.append(post_dist.mean)
            post_stds.append(post_dist.stddev)
            obs_means.append(obs_dist.mean)
            obs_stds.append(obs_dist.stddev)
            rewards.append(reward)
            
            state = {'deter': deter, 'stoch': stoch}
        
        return {
            'prior_mean': torch.stack(prior_means, dim=1),
            'prior_std': torch.stack(prior_stds, dim=1),
            'post_mean': torch.stack(post_means, dim=1),
            'post_std': torch.stack(post_stds, dim=1),
            'obs_mean': torch.stack(obs_means, dim=1),
            'obs_std': torch.stack(obs_stds, dim=1),
            'reward': torch.stack(rewards, dim=1).squeeze(-1),
        }

print('ClassicalWorldModel defined.')

## 6B.11 Multi-Seed Experiment

In [ ]:
def run_experiment(seed: int, config: Dict) -> Dict:
    """Run single experiment with given seed."""
    print(f'\n{"="*60}')
    print(f'Running experiment with seed {seed}')
    print(f'{"="*60}')
    
    # Set seeds
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Collect data
    episodes = collect_episodes('CartPole-v1', config['num_episodes'], seed=seed)
    
    # Prepare buffer
    buffer = SuperpositionReplayBuffer(capacity=1000, parallel_samples=4)
    for ep in episodes:
        buffer.add_episode(ep)
    
    # Prepare test data
    test_episodes = episodes[-10:]
    test_obs = torch.FloatTensor(np.stack([ep.observations[:config['seq_len']] for ep in test_episodes])).to(DEVICE)
    test_actions = torch.FloatTensor(np.stack([ep.actions[:config['seq_len']] for ep in test_episodes])).to(DEVICE)
    test_rewards = torch.FloatTensor(np.stack([ep.rewards[:config['seq_len']] for ep in test_episodes])).to(DEVICE)
    
    results = {}
    
    # ============================================================
    # Train Classical Baseline
    # ============================================================
    print('\nTraining Classical Baseline...')
    classical_model = ClassicalWorldModel(config['obs_dim'], config['action_dim'], config).to(DEVICE)
    classical_optimizer = torch.optim.Adam(classical_model.parameters(), lr=config['learning_rate'])
    
    classical_start = time.time()
    classical_losses = []
    
    for epoch in range(config['num_epochs']):
        classical_model.train()
        obs, act, rew = buffer.sample_standard(config['batch_size'], config['seq_len'])
        
        classical_optimizer.zero_grad()
        outputs = classical_model(obs, act)
        
        recon_loss = F.mse_loss(outputs['obs_mean'], obs)
        prior_dist = torch.distributions.Normal(outputs['prior_mean'], outputs['prior_std'])
        post_dist = torch.distributions.Normal(outputs['post_mean'], outputs['post_std'])
        kl_loss = torch.clamp(torch.distributions.kl_divergence(post_dist, prior_dist).mean(), min=1.0)
        reward_loss = F.mse_loss(outputs['reward'], rew)
        loss = recon_loss + 0.1 * kl_loss + reward_loss
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(classical_model.parameters(), 100.0)
        classical_optimizer.step()
        
        classical_losses.append(loss.item())
        
        if (epoch + 1) % 25 == 0:
            print(f'  Epoch {epoch+1}: Loss = {loss.item():.4f}')
    
    classical_time = time.time() - classical_start
    
    # Evaluate classical
    classical_model.eval()
    with torch.no_grad():
        classical_outputs = classical_model(test_obs, test_actions)
        classical_mse = F.mse_loss(classical_outputs['obs_mean'], test_obs).item()
    
    classical_horizon = evaluate_long_horizon(classical_model, test_obs, test_actions)
    
    results['classical'] = {
        'final_loss': classical_losses[-1],
        'test_mse': classical_mse,
        'training_time': classical_time,
        'long_horizon': classical_horizon,
        'params': sum(p.numel() for p in classical_model.parameters()),
    }
    
    # ============================================================
    # Train Fully Integrated
    # ============================================================
    print('\nTraining Fully Integrated (QAOA + Superposition + Gates + Ensemble)...')
    
    integrated_ensemble = ErrorCorrectionEnsemble(
        config['obs_dim'], config['action_dim'], config, num_models=config['num_ensemble']
    ).to(DEVICE)
    
    integrated_trainer = FullyIntegratedTrainer(integrated_ensemble, buffer, config)
    
    integrated_start = time.time()
    integrated_history = integrated_trainer.train(config['num_epochs'], use_superposition=True)
    integrated_time = time.time() - integrated_start
    
    # Evaluate integrated
    integrated_eval = integrated_trainer.evaluate(test_obs, test_actions)
    integrated_horizon = evaluate_long_horizon(integrated_ensemble, test_obs, test_actions)
    
    results['integrated'] = {
        'final_loss': integrated_history['total_loss'][-1],
        'test_mse': integrated_eval['ensemble_mse'],
        'training_time': integrated_time,
        'long_horizon': integrated_horizon,
        'uncertainty': integrated_eval['mean_uncertainty'],
        'params': sum(p.numel() for p in integrated_ensemble.parameters()),
    }
    
    print(f'\nSeed {seed} Results:')
    print(f'  Classical - Loss: {results["classical"]["final_loss"]:.4f}, MSE: {results["classical"]["test_mse"]:.6f}')
    print(f'  Integrated - Loss: {results["integrated"]["final_loss"]:.4f}, MSE: {results["integrated"]["test_mse"]:.6f}')
    
    return results

# Run experiments
all_results = []
for seed in CONFIG['seeds']:
    result = run_experiment(seed, CONFIG)
    all_results.append(result)

## 6B.12 Statistical Analysis

In [ ]:
print('='*70)
print('STATISTICAL ANALYSIS')
print('='*70)

# Extract metrics
classical_losses = [r['classical']['final_loss'] for r in all_results]
integrated_losses = [r['integrated']['final_loss'] for r in all_results]
classical_mses = [r['classical']['test_mse'] for r in all_results]
integrated_mses = [r['integrated']['test_mse'] for r in all_results]
classical_times = [r['classical']['training_time'] for r in all_results]
integrated_times = [r['integrated']['training_time'] for r in all_results]

# Long-horizon metrics
classical_horizon_growth = [r['classical']['long_horizon'].get('error_growth_rate', 1.0) for r in all_results]
integrated_horizon_growth = [r['integrated']['long_horizon'].get('error_growth_rate', 1.0) for r in all_results]

print('\n' + '-'*70)
print('1. FINAL LOSS COMPARISON')
print('-'*70)
print(f'Classical:  {np.mean(classical_losses):.4f} +/- {np.std(classical_losses):.4f}')
print(f'Integrated: {np.mean(integrated_losses):.4f} +/- {np.std(integrated_losses):.4f}')

# Mann-Whitney U test
stat, p_loss = stats.mannwhitneyu(classical_losses, integrated_losses, alternative='two-sided')
print(f'Mann-Whitney U: p={p_loss:.6f}')

# Cohen's d
pooled_std = np.sqrt((np.std(classical_losses)**2 + np.std(integrated_losses)**2) / 2)
cohens_d = (np.mean(classical_losses) - np.mean(integrated_losses)) / pooled_std
print(f"Cohen's d: {cohens_d:.4f}")

print('\n' + '-'*70)
print('2. TEST MSE COMPARISON')
print('-'*70)
print(f'Classical:  {np.mean(classical_mses):.6f} +/- {np.std(classical_mses):.6f}')
print(f'Integrated: {np.mean(integrated_mses):.6f} +/- {np.std(integrated_mses):.6f}')

stat, p_mse = stats.mannwhitneyu(classical_mses, integrated_mses, alternative='two-sided')
print(f'Mann-Whitney U: p={p_mse:.6f}')

improvement = (np.mean(classical_mses) - np.mean(integrated_mses)) / np.mean(classical_mses) * 100
print(f'Improvement: {improvement:.1f}%')

print('\n' + '-'*70)
print('3. LONG-HORIZON PREDICTION (Error Growth Rate)')
print('-'*70)
print(f'Classical error growth:  {np.mean(classical_horizon_growth):.4f} +/- {np.std(classical_horizon_growth):.4f}')
print(f'Integrated error growth: {np.mean(integrated_horizon_growth):.4f} +/- {np.std(integrated_horizon_growth):.4f}')
print('(Lower is better - errors compound slower)')

print('\n' + '-'*70)
print('4. TRAINING TIME')
print('-'*70)
print(f'Classical:  {np.mean(classical_times):.2f}s +/- {np.std(classical_times):.2f}s')
print(f'Integrated: {np.mean(integrated_times):.2f}s +/- {np.std(integrated_times):.2f}s')

print('\n' + '-'*70)
print('5. COMPUTATIONAL COST')
print('-'*70)
print(f'Classical params:  {all_results[0]["classical"]["params"]:,}')
print(f'Integrated params: {all_results[0]["integrated"]["params"]:,}')
print(f'Ratio: {all_results[0]["integrated"]["params"] / all_results[0]["classical"]["params"]:.1f}x')

## 6B.13 Save Results

In [ ]:
# Save results
results_dir = Path('../experiments/results/fully_integrated')
results_dir.mkdir(parents=True, exist_ok=True)

# Prepare results dict
save_results = {
    'metadata': {
        'experiment': 'Fully Integrated Quantum-Enhanced World Model',
        'seeds': CONFIG['seeds'],
        'num_epochs': CONFIG['num_epochs'],
        'environment': 'CartPole-v1',
    },
    'classical': {
        'final_losses': classical_losses,
        'test_mses': classical_mses,
        'training_times': classical_times,
        'mean_loss': float(np.mean(classical_losses)),
        'std_loss': float(np.std(classical_losses)),
        'mean_mse': float(np.mean(classical_mses)),
        'std_mse': float(np.std(classical_mses)),
    },
    'integrated': {
        'final_losses': integrated_losses,
        'test_mses': integrated_mses,
        'training_times': integrated_times,
        'mean_loss': float(np.mean(integrated_losses)),
        'std_loss': float(np.std(integrated_losses)),
        'mean_mse': float(np.mean(integrated_mses)),
        'std_mse': float(np.std(integrated_mses)),
    },
    'statistical_tests': {
        'loss_p_value': float(p_loss),
        'mse_p_value': float(p_mse),
        'cohens_d': float(cohens_d),
        'mse_improvement_percent': float(improvement),
    },
    'long_horizon': {
        'classical_error_growth': classical_horizon_growth,
        'integrated_error_growth': integrated_horizon_growth,
    },
}

with open(results_dir / 'complete_metrics.json', 'w') as f:
    json.dump(save_results, f, indent=2)

print(f'Results saved to {results_dir / "complete_metrics.json"}')

## 6B.14 Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Final Loss Comparison
ax = axes[0, 0]
x = np.arange(len(CONFIG['seeds']))
width = 0.35
ax.bar(x - width/2, classical_losses, width, label='Classical', color='steelblue', alpha=0.8)
ax.bar(x + width/2, integrated_losses, width, label='Integrated', color='darkorange', alpha=0.8)
ax.set_xlabel('Seed')
ax.set_ylabel('Final Loss')
ax.set_title('Final Loss by Seed')
ax.set_xticks(x)
ax.set_xticklabels(CONFIG['seeds'])
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Test MSE Comparison
ax = axes[0, 1]
ax.bar(x - width/2, classical_mses, width, label='Classical', color='steelblue', alpha=0.8)
ax.bar(x + width/2, integrated_mses, width, label='Integrated', color='darkorange', alpha=0.8)
ax.set_xlabel('Seed')
ax.set_ylabel('Test MSE')
ax.set_title('Test MSE by Seed')
ax.set_xticks(x)
ax.set_xticklabels(CONFIG['seeds'])
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Summary Statistics
ax = axes[1, 0]
metrics = ['Loss', 'MSE', 'Time (s)']
classical_means = [np.mean(classical_losses), np.mean(classical_mses)*100, np.mean(classical_times)]
integrated_means = [np.mean(integrated_losses), np.mean(integrated_mses)*100, np.mean(integrated_times)]
x = np.arange(len(metrics))
ax.bar(x - width/2, classical_means, width, label='Classical', color='steelblue', alpha=0.8)
ax.bar(x + width/2, integrated_means, width, label='Integrated', color='darkorange', alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.set_title('Summary Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Long-Horizon Error Growth
ax = axes[1, 1]
ax.bar(x[:len(classical_horizon_growth)] - width/2, classical_horizon_growth, width, 
       label='Classical', color='steelblue', alpha=0.8)
ax.bar(x[:len(integrated_horizon_growth)] + width/2, integrated_horizon_growth, width, 
       label='Integrated', color='darkorange', alpha=0.8)
ax.set_xlabel('Seed Index')
ax.set_ylabel('Error Growth Rate')
ax.set_title('Long-Horizon Error Growth (Lower is Better)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(results_dir / 'comparison_plots.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Figure saved to {results_dir / "comparison_plots.png"}')

## 6B.15 Summary

### What This Notebook Combines

| Component | From | Solves |
|-----------|------|--------|
| QAOA Optimizer | Notebook 03 | Local minima |
| Superposition Replay | Notebook 04 | Sample inefficiency |
| Gate-Enhanced Layers | Notebook 05 | Slow convergence |
| Error Correction Ensemble | Notebook 06 | Compounding errors |

### Key Metrics Measured

1. **Training Efficiency**: Wall-clock time
2. **Sample Efficiency**: Final loss with same data
3. **Final Performance**: Test MSE
4. **Long-Horizon Prediction**: Error growth rate
5. **Computational Cost**: Parameter count
6. **Statistical Significance**: Mann-Whitney U, Cohen's d

In [ ]:
print('='*70)
print('Phase 6B: Fully Integrated - COMPLETE')
print('='*70)
print()
print('Components integrated:')
print('  - QAOA Optimizer (escapes local minima)')
print('  - Superposition Replay (sample efficiency)')
print('  - Gate-Enhanced Layers (better representations)')
print('  - Error Correction Ensemble (robust predictions)')
print()
print('Metrics measured:')
print('  - Training efficiency (time)')
print('  - Sample efficiency (loss)')
print('  - Final performance (MSE)')
print('  - Long-horizon prediction (error growth)')
print('  - Computational cost (params)')
print('  - Statistical significance (p-values)')
print()
print('Ready for comparison with individual approaches!')